In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
import mlflow
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from typing import List, Tuple, Any, Dict
from enum import Enum
from sklearn.base import BaseEstimator, TransformerMixin
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.model_selection import (
    StratifiedKFold,
)

from mlflow.models.signature import infer_signature
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    confusion_matrix,
    balanced_accuracy_score,
)
from sklearn.preprocessing import PolynomialFeatures

from sklearn.metrics import classification_report
from functools import partial
import optuna
import lightgbm as lgb

HOST = "127.0.0.1"
PORT = "8080"
mlflow.set_tracking_uri(uri=f"http://{HOST}:{PORT}")


## Import données

In [14]:
data = pd.read_csv("./data/application_train.csv")

print(data.shape)

data.head()


(307511, 122)


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
def _check_most_frequent_values(
    df: pd.DataFrame, columns: list, col_type: str
) -> None:
    """Check and warn about multiple most frequent values in categorical columns"""
    for col in columns:
        value_counts = df[col].value_counts()
        if (value_counts == value_counts.iloc[0]).sum() > 1:
            print(
                f"Warning: Multiple most frequent values in {col_type} column {col}"
            )
            print(
                f"Values: {list(value_counts[value_counts == value_counts.iloc[0]].index)}"
            )


def get_column_types(
    df: pd.DataFrame,
) -> Tuple[List[str], List[str], List[str]]:
    """
    Identify and categorize column types based on data characteristics.

    Args:
        df: Input DataFrame

    Returns:
        Tuple containing lists of numerical, binary and multiclass column names
    """
    numerical_columns = df.select_dtypes(
        include=["int64", "float64"]
    ).columns.tolist()
    categorical_columns = df.select_dtypes(
        include=["object", "category"]
    ).columns.tolist()

    binary_columns = [
        col for col in categorical_columns if df[col].nunique() <= 2
    ]
    multiclass_columns = [
        col for col in categorical_columns if df[col].nunique() > 2
    ]

    print(f"Found {len(numerical_columns)} numerical features")
    print(f"Found {len(binary_columns)} binary features")
    print(f"Found {len(multiclass_columns)} categorical features")

    return numerical_columns, binary_columns, multiclass_columns


class OutlierStrategy(str, Enum):
    KEEP = "keep"
    REPLACE_WITH_NAN = "replace_with_nan"
    DROP = "drop"


class OutlierHandler(BaseEstimator, TransformerMixin):
    def __init__(
        self,
        strategy: OutlierStrategy = OutlierStrategy.KEEP,
        threshold: float = 0.1,
        columns: List[str] = None,
    ):
        self.strategy = strategy
        self.threshold = threshold
        self.columns = columns
        self.lower_bounds_ = {}
        self.upper_bounds_ = {}
        self.to_drop_idx_ = None

    def fit(self, X, y=None):
        X = pd.DataFrame(X)
        cols = self.columns or X.select_dtypes(include=["number"]).columns

        for column in cols:
            quantiles = X[column].quantile([self.threshold, 1 - self.threshold])
            self.lower_bounds_[column] = quantiles.iloc[0]
            self.upper_bounds_[column] = quantiles.iloc[1]
        return self

    def transform(self, X):
        X = pd.DataFrame(X).copy()
        cols = self.columns or X.select_dtypes(include=["number"]).columns

        if self.strategy == OutlierStrategy.KEEP:
            return X

        mask = np.ones(len(X), dtype=bool)

        for column in cols:
            outlier_mask = (X[column] < self.lower_bounds_[column]) | (
                X[column] > self.upper_bounds_[column]
            )
            if self.strategy == OutlierStrategy.REPLACE_WITH_NAN:
                X.loc[outlier_mask, column] = np.nan
            elif self.strategy == OutlierStrategy.DROP:
                mask &= ~outlier_mask

        if self.strategy == OutlierStrategy.DROP:
            self.to_drop_idx_ = X.index[~mask]
            X = X[mask]

        return X


class DropMissingColumns(BaseEstimator, TransformerMixin):
    def __init__(self, threshold=0.4):
        """
        Drops columns with missing value rate >= threshold.
        """
        self.threshold = threshold
        self.cols_to_drop_ = []

    def fit(self, X, y=None):
        X = pd.DataFrame(X)
        missing_ratio = X.isnull().mean()
        self.cols_to_drop_ = missing_ratio[
            missing_ratio >= self.threshold
        ].index.tolist()

        if self.cols_to_drop_:
            print(
                f"🧹 Dropping {len(self.cols_to_drop_)} column(s) with missing ratio ≥ {self.threshold}:"
            )
            for col in self.cols_to_drop_:
                print(f" - {col}")

        return self

    def transform(self, X):
        return pd.DataFrame(X).drop(columns=self.cols_to_drop_, errors="ignore")

    @property
    def removed_columns_(self):
        return self.cols_to_drop_


def create_preprocessing_pipeline(
    df: pd.DataFrame,
    numerical_cols: List[str],
    binary_cols: List[str],
    multiclass_cols: List[str],
    categorical_strategy: str,
    encode_categoricals: bool = True,
    polynomial_features: bool = False,
) -> ColumnTransformer:
    """
    Creates a preprocessing pipeline for numerical, binary, and multiclass categorical features.

    Args:
        df (pd.DataFrame): The full dataframe used for checking value consistency if needed.
        numerical_cols (List[str]): List of numerical feature column names.
        binary_cols (List[str]): List of binary categorical feature column names.
        multiclass_cols (List[str]): List of multi-category feature column names.
        categorical_strategy (str): Strategy for imputing categorical values ("most_frequent", "constant", etc.).
        encode_categoricals (bool): Whether to encode categorical features.

    Returns:
        ColumnTransformer: A scikit-learn column transformer pipeline.
    """

    transformers = []

    # --- Numerical columns ---
    if numerical_cols:
        steps = [("imputer", SimpleImputer(strategy="median"))]
        if polynomial_features:
            steps.append(
                (
                    "poly",
                    PolynomialFeatures(
                        degree=3, include_bias=False, interaction_only=False
                    ),
                )
            )
        steps.append(("scaler", MinMaxScaler()))

        numerical_pipeline = Pipeline(steps)
        transformers.append(("numerical", numerical_pipeline, numerical_cols))

    # --- Categorical encoding ---
    if encode_categoricals:
        # --- Binary columns ---
        if binary_cols:
            if categorical_strategy == "most_frequent":
                _check_most_frequent_values(df, binary_cols, "binary")

            binary_pipeline = Pipeline(
                [
                    (
                        "imputer",
                        SimpleImputer(
                            strategy=categorical_strategy,
                            fill_value="missing"
                            if categorical_strategy == "constant"
                            else None,
                        ),
                    ),
                    ("encoder", OrdinalEncoder()),
                ]
            )
            transformers.append(("binary", binary_pipeline, binary_cols))

        # --- Multiclass columns ---
        if multiclass_cols:
            if categorical_strategy == "most_frequent":
                _check_most_frequent_values(df, multiclass_cols, "multiclass")

            multiclass_pipeline = Pipeline(
                [
                    (
                        "imputer",
                        SimpleImputer(
                            strategy=categorical_strategy,
                            fill_value="missing"
                            if categorical_strategy == "constant"
                            else None,
                        ),
                    ),
                    (
                        "onehot",
                        OneHotEncoder(
                            drop="first",
                            sparse_output=False,
                            handle_unknown="ignore",
                        ),
                    ),
                ]
            )
            transformers.append(
                ("multiclass", multiclass_pipeline, multiclass_cols)
            )

    preprocessor = ColumnTransformer(transformers)
    return preprocessor


def prepare_data(
    df: pd.DataFrame,
    target_column: str,
    categorical_strategy: str,
    outlier_strategy: str,
    outlier_threshold: float = 0.1,
    test_size: float = 0.2,
    random_state: int = 42,
    encode_categoricals: bool = True,
    drop_missing_columns: bool = True,
    missing_col_threshold: float = 0.4,
    polynomial_features: bool = False,
):
    """Prepare data for modeling using pipeline approach"""

    X = df.drop(columns=[target_column])
    y = df[target_column]

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state, stratify=y
    )

    # Manage missing values
    if drop_missing_columns:
        dropper = DropMissingColumns(threshold=missing_col_threshold)
        X_train = dropper.fit_transform(X_train)
        X_test = dropper.transform(X_test)

    numerical_cols, binary_cols, multiclass_cols = get_column_types(X_train)

    if outlier_strategy != OutlierStrategy.KEEP:
        outlier_handler = OutlierHandler(
            strategy=outlier_strategy,
            threshold=outlier_threshold,
            columns=numerical_cols,
        )
        X_train = outlier_handler.fit_transform(X_train)

        if (
            outlier_strategy == OutlierStrategy.DROP
            and outlier_handler.to_drop_idx_ is not None
        ):
            kept_index = X_train.index
            X_train = X_train
            y_train = y_train.loc[kept_index]
        else:
            X_train = X_train

    preprocessor = create_preprocessing_pipeline(
        X_train,
        numerical_cols,
        binary_cols,
        multiclass_cols,
        categorical_strategy=categorical_strategy,
        encode_categoricals=encode_categoricals,
        polynomial_features=polynomial_features,
    )

    # X_train_processed = preprocessor.fit_transform(X_train)
    # X_test_processed = preprocessor.transform(X_test)

    return X_train, X_test, y_train, y_test, preprocessor


# Test pipeline

In [16]:
categorical_strategies = ["constant", "most_frequent"]
outlier_strategies = [
    OutlierStrategy.KEEP,
    OutlierStrategy.REPLACE_WITH_NAN,
    OutlierStrategy.DROP,
]

results = {}

for cat_strategy in categorical_strategies:
    for out_strategy in outlier_strategies:
        print("\nTesting combination:")
        print(f"Categorical strategy: {cat_strategy}")
        print(f"Outlier strategy: {out_strategy}")

        X_train_proc, X_test_proc, y_train_proc, y_test_proc, preproc = (
            prepare_data(
                data,
                target_column="TARGET",
                categorical_strategy=cat_strategy,
                outlier_strategy=out_strategy,
                outlier_threshold=0.05,
                test_size=0.2,
                random_state=42,
            )
        )

        results[(cat_strategy, out_strategy)] = {
            "X_train_shape": X_train_proc.shape,
            "X_test_shape": X_test_proc.shape,
            "success": True,
        }

        print("Shapes après transformation:")
        print(f"X_train: {X_train_proc.shape}")
        print(f"X_test: {X_test_proc.shape}")

print("\nRésumé des tests:")
print("-" * 50)
for (cat_strategy, out_strategy), result in results.items():
    print(f"\nCategorical: {cat_strategy}")
    print(f"Outlier: {out_strategy}")
    if result["success"]:
        print("✓ Success")
        print(f"  Train shape: {result['X_train_shape']}")
        print(f"  Test shape: {result['X_test_shape']}")
    else:
        print("✗ Failed")
        print(f"  Error: {result['error']}")



Testing combination:
Categorical strategy: constant
Outlier strategy: OutlierStrategy.KEEP
🧹 Dropping 49 column(s) with missing ratio ≥ 0.4:
 - OWN_CAR_AGE
 - EXT_SOURCE_1
 - APARTMENTS_AVG
 - BASEMENTAREA_AVG
 - YEARS_BEGINEXPLUATATION_AVG
 - YEARS_BUILD_AVG
 - COMMONAREA_AVG
 - ELEVATORS_AVG
 - ENTRANCES_AVG
 - FLOORSMAX_AVG
 - FLOORSMIN_AVG
 - LANDAREA_AVG
 - LIVINGAPARTMENTS_AVG
 - LIVINGAREA_AVG
 - NONLIVINGAPARTMENTS_AVG
 - NONLIVINGAREA_AVG
 - APARTMENTS_MODE
 - BASEMENTAREA_MODE
 - YEARS_BEGINEXPLUATATION_MODE
 - YEARS_BUILD_MODE
 - COMMONAREA_MODE
 - ELEVATORS_MODE
 - ENTRANCES_MODE
 - FLOORSMAX_MODE
 - FLOORSMIN_MODE
 - LANDAREA_MODE
 - LIVINGAPARTMENTS_MODE
 - LIVINGAREA_MODE
 - NONLIVINGAPARTMENTS_MODE
 - NONLIVINGAREA_MODE
 - APARTMENTS_MEDI
 - BASEMENTAREA_MEDI
 - YEARS_BEGINEXPLUATATION_MEDI
 - YEARS_BUILD_MEDI
 - COMMONAREA_MEDI
 - ELEVATORS_MEDI
 - ENTRANCES_MEDI
 - FLOORSMAX_MEDI
 - FLOORSMIN_MEDI
 - LANDAREA_MEDI
 - LIVINGAPARTMENTS_MEDI
 - LIVINGAREA_MEDI
 - NONLIV

In [17]:
X_train_proc, X_test_proc, y_train_proc, y_test_proc, preproc = prepare_data(
    data[
        ["EXT_SOURCE_1", "EXT_SOURCE_2", "EXT_SOURCE_3", "DAYS_BIRTH", "TARGET"]
    ],
    target_column="TARGET",
    categorical_strategy="most_frequent",
    outlier_strategy="keep",
    outlier_threshold=0.05,
    test_size=0.2,
    random_state=42,
    encode_categoricals=False,
    drop_missing_columns=True,
    missing_col_threshold=0.6,
    polynomial_features=True,
)
X_train_proc


Found 4 numerical features
Found 0 binary features
Found 0 categorical features


,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,DAYS_BIRTH
181648,0.384582,0.289573,0.622922,-12298
229245,NaN,0.514261,NaN,-15375
122525,NaN,0.486906,0.598926,-19307
306311,NaN,0.675705,0.454321,-17791
300658,NaN,0.154565,NaN,-8486
...,...,...,...,...
31304,NaN,0.549668,NaN,-15374
121193,NaN,0.569702,0.600658,-19035
248504,NaN,0.461966,0.683269,-23088
175469,NaN,0.459173,0.812823,-22148


# MLFLOW

In [18]:
def plot_confusion_matrix_heatmap(cm, labels, title="Confusion Matrix"):
    plt.figure(figsize=(6, 4))
    sns.heatmap(
        cm,
        annot=True,
        fmt="d",
        cmap="Blues",
        xticklabels=labels,
        yticklabels=labels,
    )
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.title(title)
    plt.tight_layout()
    img_path = "confusion_matrix_heatmap.png"
    plt.savefig(img_path)
    plt.close()
    return img_path


def evaluate_model(y_true, y_pred, y_proba):
    cm = confusion_matrix(y_true, y_pred)
    metrics = {
        "accuracy": accuracy_score(y_true, y_pred),
        "balanced_accuracy": balanced_accuracy_score(y_true, y_pred),
        "precision": precision_score(y_true, y_pred),
        "recall": recall_score(y_true, y_pred),
        "f1": f1_score(y_true, y_pred),
        "roc_auc": roc_auc_score(y_true, y_proba),
        "combined_score": 0.7 * recall_score(y_true, y_pred)
        + 0.3 * precision_score(y_true, y_pred),
    }
    return metrics, cm


def log_mlflow_models(
    df: pd.DataFrame, pipeline: Any, artifact_path: str
) -> None:
    """
    Logs a trained scikit-learn pipeline to MLflow, including input signature and example.

    Args:
        df (pd.DataFrame): Input features used to infer the model signature and create an input example.
        pipeline (Any): Trained scikit-learn pipeline to log.
        artifact_path (str): Path in the MLflow run where the model will be saved (e.g., "model", "model_final").

    Returns:
        None
    """

    signature = infer_signature(df, pipeline.predict(df))
    input_example = df.dropna().iloc[:5]

    mlflow.sklearn.log_model(
        pipeline,
        artifact_path=artifact_path,
        signature=signature,
        input_example=input_example,
    )


In [33]:
def run_experiment(
    model_name,
    model_class,
    param_space_fn,
    dataset_name,
    X_train,
    X_test,
    y_train,
    y_test,
    preprocessor,
    cv=5,
    n_trials=30,
):
    experiment_name = f"{model_name}"
    mlflow.set_experiment(experiment_name)

    with mlflow.start_run(run_name=f"{model_name}__{dataset_name}__study_run"):
        mlflow.set_tag("model_type", model_name)
        mlflow.set_tag("dataset", dataset_name)
        mlflow.set_tag("experiment_level", "test")

        def objective(trial):
            trial_params = param_space_fn(trial)
            threshold = trial.suggest_float("threshold", 0.1, 0.5, step=0.01)

            with mlflow.start_run(nested=True):
                mlflow.set_tag("model_type", model_name)
                mlflow.set_tag("dataset", dataset_name)
                mlflow.set_tag("experiment_level", "trial")
                mlflow.set_tag("trial_number", trial.number)
                mlflow.log_param("threshold", threshold)
                mlflow.log_param("cv_folds", cv)
                mlflow.log_param("dataset_name", dataset_name)
                mlflow.log_param("n_trials", n_trials)

                for param_name, param_value in trial_params.items():
                    if param_name == "classifier__class_weight" and isinstance(
                        param_value, dict
                    ):
                        mlflow.log_param("pos_weight", param_value.get(1))
                    else:
                        clean_name = param_name.replace("classifier__", "")
                        mlflow.log_param(clean_name, param_value)

                pipeline = Pipeline(
                    [
                        ("preprocessing", preprocessor),
                        ("classifier", model_class(random_state=42, n_jobs=1)),
                    ]
                )
                pipeline.set_params(**trial_params)

                kf = StratifiedKFold(n_splits=cv, shuffle=True, random_state=42)
                recall_scores = []
                precision_scores = []
                accuracy_scores = []
                f1_scores = []
                balanced_accuracy_scores = []
                roc_auc_scores = []

                for fold_idx, (train_idx, val_idx) in enumerate(
                    kf.split(X_train, y_train)
                ):
                    X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
                    y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

                    pipeline.fit(X_tr, y_tr)
                    y_proba = pipeline.predict_proba(X_val)[:, 1]
                    y_pred = (y_proba > threshold).astype(int)

                    recall = recall_score(y_val, y_pred)
                    precision = precision_score(y_val, y_pred)
                    accuracy = accuracy_score(y_val, y_pred)
                    f1 = f1_score(y_val, y_pred)
                    balanced_accuracy = balanced_accuracy_score(y_val, y_pred)
                    roc_auc = roc_auc_score(y_val, y_proba)

                    recall_scores.append(recall)
                    precision_scores.append(precision)
                    accuracy_scores.append(accuracy)
                    f1_scores.append(f1)
                    balanced_accuracy_scores.append(balanced_accuracy)
                    roc_auc_scores.append(roc_auc)

                    # # Optuna pruner: report step-wise performance
                    # trial.report(f1, step=fold_idx)

                    # if trial.should_prune():
                    #     mlflow.log_param("pruned_at_step", fold_idx)
                    #     mlflow.log_metric("partial_f1", f1)
                    #     print(
                    #         f"⏹️ Trial {trial.number} pruned at fold {fold_idx} (f1={f1:.4f})"
                    #     )
                    #     raise optuna.exceptions.TrialPruned()

                avg_recall = np.mean(recall_scores)
                avg_precision = np.mean(precision_scores)
                avg_accuracy = np.mean(accuracy_scores)
                avg_f1 = np.mean(f1_scores)
                avg_balanced_accuracy = np.mean(balanced_accuracy_scores)
                avg_roc_auc = np.mean(roc_auc_scores)

                mlflow.log_metric("recall", avg_recall)
                mlflow.log_metric("precision", avg_precision)
                mlflow.log_metric("accuracy", avg_accuracy)
                mlflow.log_metric("f1", avg_f1)
                mlflow.log_metric("balanced_accuracy", avg_balanced_accuracy)

                combined_score = 0.8 * avg_recall + 0.2 * avg_precision
                mlflow.log_metric("combined_score", combined_score)
                mlflow.log_metric("roc_auc", avg_roc_auc)

                log_mlflow_models(
                    X_train,
                    pipeline,
                    "model",
                )

                return combined_score

        # pruner=optuna.pruners.MedianPruner(n_startup_trials=5)
        study = optuna.create_study(direction="maximize")
        study.optimize(objective, n_trials=n_trials, n_jobs=1)

        best_params = study.best_params
        best_threshold = best_params.pop("threshold")
        solver_penalty = study.best_params.get("solver_penalty")
        class_weight = study.best_params.get("positive_class_weight")

        if solver_penalty:
            solver, penalty = solver_penalty.split("__")
        best_params = {
            k: v for k, v in best_params.items() if k.startswith("classifier__")
        }
        # Récupère les paramètres globaux

        mlflow.log_param("cv_folds", cv)
        mlflow.log_param("dataset_name", dataset_name)
        mlflow.log_param("n_trials", n_trials)
        mlflow.log_param("threshold", best_threshold)
        for param_name, param_value in best_params.items():
            clean_name = param_name.replace("classifier__", "")
            mlflow.log_param(clean_name, param_value)
        if model_class == LogisticRegression:
            mlflow.log_param("solver", solver)
            mlflow.log_param("penalty", penalty)
        mlflow.log_param("pos_weight", class_weight)

        pipeline = Pipeline(
            [
                ("preprocessing", preprocessor),
                ("classifier", model_class(random_state=42, n_jobs=1)),
            ]
        )
        pipeline.set_params(**best_params)
        pipeline.fit(X_train, y_train)

        y_proba = pipeline.predict_proba(X_test)[:, 1]
        y_pred = (y_proba > best_threshold).astype(int)

        metrics, cm = evaluate_model(y_test, y_pred, y_proba)
        for key, value in metrics.items():
            mlflow.log_metric(key, value)

        img_path = plot_confusion_matrix_heatmap(
            cm, labels=[0, 1], title=experiment_name
        )
        mlflow.log_artifact(img_path)
        os.remove(img_path)

        report_str = classification_report(y_test, y_pred)
        report_path_txt = "classification_report.txt"
        with open(report_path_txt, "w") as f:
            f.write(report_str)
        mlflow.log_artifact(report_path_txt)
        os.remove(report_path_txt)

        log_mlflow_models(
            X_train,
            pipeline,
            "model_final",
        )


# Param space

In [20]:
def xgboost_param_space(
    trial: optuna.trial.Trial,
    n_estimators_range: Tuple[int, int],
    max_depth_range: Tuple[int, int],
    learning_rate_range: Tuple[float, float],
    subsample_range: Tuple[float, float],
    colsample_bytree_range: Tuple[float, float],
    gamma_range: Tuple[float, float],
    reg_alpha_range: Tuple[float, float],
    reg_lambda_range: Tuple[float, float],
    scale_pos_weight_range: Tuple[float, float],
) -> Dict[str, Any]:
    """
    Suggests a hyperparameter configuration for an XGBoost classifier using Optuna.

    Args:
        trial: Optuna trial object used to sample hyperparameters.
        n_estimators_range: Range for the number of boosting rounds.
        max_depth_range: Range for maximum tree depth.
        learning_rate_range: Range for learning rate (log scale recommended).
        subsample_range: Range for row subsampling.
        colsample_bytree_range: Range for feature subsampling.
        gamma_range: Range for minimum loss reduction to split a node.
        reg_alpha_range: L1 regularization term.
        reg_lambda_range: L2 regularization term.
        scale_pos_weight_range: Range for the ratio of negative to positive samples (class imbalance correction).

    Returns:
        A dictionary of hyperparameters for a scikit-learn Pipeline using XGBClassifier.
    """
    params = {
        "classifier__n_estimators": trial.suggest_int(
            "classifier__n_estimators", *n_estimators_range
        ),
        "classifier__max_depth": trial.suggest_int(
            "classifier__max_depth", *max_depth_range
        ),
        "classifier__learning_rate": trial.suggest_float(
            "classifier__learning_rate", *learning_rate_range, log=True
        ),
        "classifier__subsample": trial.suggest_float(
            "classifier__subsample", *subsample_range
        ),
        "classifier__colsample_bytree": trial.suggest_float(
            "classifier__colsample_bytree", *colsample_bytree_range
        ),
        "classifier__gamma": trial.suggest_float(
            "classifier__gamma", *gamma_range
        ),
        "classifier__reg_alpha": trial.suggest_float(
            "classifier__reg_alpha", *reg_alpha_range
        ),
        "classifier__reg_lambda": trial.suggest_float(
            "classifier__reg_lambda", *reg_lambda_range
        ),
        "classifier__scale_pos_weight": trial.suggest_float(
            "classifier__scale_pos_weight", *scale_pos_weight_range
        ),
    }
    return params


# Run

## Logistic  Regression


In [21]:
def logistic_regression_param_space(
    trial: optuna.trial.Trial,
    class_weight_range: Tuple[float, float],
    max_iter_range: Tuple[int, int],
    tol_range: Tuple[float, float],
    C_range: Tuple[float, float],
    l1_ratio_range: Tuple[float, float],
) -> Dict[str, Any]:
    """
    Suggests a hyperparameter configuration for a LogisticRegression classifier using Optuna.

    Args:
        trial: Optuna trial object used to sample hyperparameters.
        class_weight_range: Range for the positive class weight (e.g., (1, 100)).
        max_iter_range: Range for the maximum number of iterations.
        tol_range: Range for solver tolerance (log scale recommended).
        C_range: Range for the inverse of regularization strength (log scale).
        l1_ratio_range: Range for L1 mixing ratio (only used if penalty='elasticnet').

    Returns:
        A dictionary of hyperparameters with keys prefixed by 'classifier__', ready to be passed to a scikit-learn Pipeline.
    """
    valid_combos = [
        "liblinear__l1",
        "liblinear__l2",
        "lbfgs__l2",
        "saga__l1",
        "saga__l2",
        "saga__elasticnet",
    ]
    combo = trial.suggest_categorical("solver_penalty", valid_combos)
    solver, penalty = combo.split("__")

    pos_weight = trial.suggest_float(
        "positive_class_weight", *class_weight_range
    )
    class_weight = {0: 1, 1: pos_weight}

    params = {
        "classifier__solver": solver,
        "classifier__penalty": penalty,
        "classifier__max_iter": trial.suggest_int(
            "classifier__max_iter", *max_iter_range
        ),
        "classifier__tol": trial.suggest_float(
            "classifier__tol", *tol_range, log=True
        ),
        "classifier__class_weight": class_weight,
    }

    if penalty is not None:
        params["classifier__C"] = trial.suggest_float(
            "classifier__C", *C_range, log=True
        )

    if penalty == "elasticnet":
        params["classifier__l1_ratio"] = trial.suggest_float(
            "classifier__l1_ratio", *l1_ratio_range
        )

    return params


lr_param_space = partial(
    logistic_regression_param_space,
    class_weight_range=(1, 100),
    max_iter_range=(100, 300),
    tol_range=(1e-4, 1e-2),
    C_range=(0.01, 10),
    l1_ratio_range=(0.0, 1.0),
)


In [22]:
X_train, X_test, y_train, y_test, preproc = prepare_data(
    data,
    target_column="TARGET",
    categorical_strategy="most_frequent",
    outlier_strategy="replace_with_nan",
    outlier_threshold=0.05,
    test_size=0.2,
    random_state=42,
    encode_categoricals=True,
    drop_missing_columns=True,
    missing_col_threshold=0.5,
    polynomial_features=False,
)

run_experiment(
    model_name="LogisticRegression_v2",
    model_class=LogisticRegression,
    param_space_fn=lr_param_space,
    dataset_name="application_train_managed_outliers_mising_cols",
    X_train=X_train,
    X_test=X_test,
    y_train=y_train,
    y_test=y_test,
    preprocessor=preproc,
    cv=5,
    n_trials=30,
)


🧹 Dropping 41 column(s) with missing ratio ≥ 0.5:
 - OWN_CAR_AGE
 - EXT_SOURCE_1
 - APARTMENTS_AVG
 - BASEMENTAREA_AVG
 - YEARS_BUILD_AVG
 - COMMONAREA_AVG
 - ELEVATORS_AVG
 - ENTRANCES_AVG
 - FLOORSMIN_AVG
 - LANDAREA_AVG
 - LIVINGAPARTMENTS_AVG
 - LIVINGAREA_AVG
 - NONLIVINGAPARTMENTS_AVG
 - NONLIVINGAREA_AVG
 - APARTMENTS_MODE
 - BASEMENTAREA_MODE
 - YEARS_BUILD_MODE
 - COMMONAREA_MODE
 - ELEVATORS_MODE
 - ENTRANCES_MODE
 - FLOORSMIN_MODE
 - LANDAREA_MODE
 - LIVINGAPARTMENTS_MODE
 - LIVINGAREA_MODE
 - NONLIVINGAPARTMENTS_MODE
 - NONLIVINGAREA_MODE
 - APARTMENTS_MEDI
 - BASEMENTAREA_MEDI
 - YEARS_BUILD_MEDI
 - COMMONAREA_MEDI
 - ELEVATORS_MEDI
 - ENTRANCES_MEDI
 - FLOORSMIN_MEDI
 - LANDAREA_MEDI
 - LIVINGAPARTMENTS_MEDI
 - LIVINGAREA_MEDI
 - NONLIVINGAPARTMENTS_MEDI
 - NONLIVINGAREA_MEDI
 - FONDKAPREMONT_MODE
 - HOUSETYPE_MODE
 - WALLSMATERIAL_MODE
Found 67 numerical features
Found 4 binary features
Found 9 categorical features


[I 2025-05-08 17:20:46,419] A new study created in memory with name: no-name-feec3b71-cf4e-4226-860f-01c7c01447c0
/Users/jeremy/code/ocr/p7_pret_a_depenser/venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 2.
  warnings.warn(
/Users/jeremy/code/ocr/p7_pret_a_depenser/venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 2.
  warnings.warn(
/Users/jeremy/code/ocr/p7_pret_a_depenser/venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 2.
  warnings.warn(
/Users/jeremy/code/ocr/p7_pret_a_depenser/venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning: 'n_jobs' > 1 does not have any effect wh

🏃 View run vaunted-wolf-202 at: http://127.0.0.1:8080/#/experiments/579090426674828470/runs/68e330edaf154ff899074f6ef0e5caf5
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/579090426674828470


/Users/jeremy/code/ocr/p7_pret_a_depenser/venv/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/jeremy/code/ocr/p7_pret_a_depenser/venv/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/jeremy/code/ocr/p7_pret_a_depenser/venv/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/jeremy/code/ocr/p7_pret_a_depenser/venv/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/jeremy/code/ocr/p7_pret_a_depenser/venv/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached w

🏃 View run dazzling-koi-576 at: http://127.0.0.1:8080/#/experiments/579090426674828470/runs/c08f268d1fce430798c78646d25d1c8a
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/579090426674828470


...

🏃 View run honorable-fox-397 at: http://127.0.0.1:8080/#/experiments/579090426674828470/runs/1cb14983caa7492f95bed1ed15746bba
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/579090426674828470


[I 2025-05-08 18:03:28,978] Trial 28 finished with value: 0.8161492956615869 and parameters: {'solver_penalty': 'lbfgs__l2', 'positive_class_weight': 48.42436926035391, 'classifier__max_iter': 259, 'classifier__tol': 0.00011706644533153366, 'classifier__C': 1.2278071166987328, 'threshold': 0.2}. Best is trial 28 with value: 0.8161492956615869.


🏃 View run gentle-eel-345 at: http://127.0.0.1:8080/#/experiments/579090426674828470/runs/421311dc069d4aabbe540315ecb5c120
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/579090426674828470


[I 2025-05-08 18:03:53,326] Trial 29 finished with value: 0.8160870886545237 and parameters: {'solver_penalty': 'lbfgs__l2', 'positive_class_weight': 42.64209245641346, 'classifier__max_iter': 258, 'classifier__tol': 0.00013028247465705976, 'classifier__C': 1.6187277598893888, 'threshold': 0.25}. Best is trial 28 with value: 0.8161492956615869.


🏃 View run mysterious-wren-735 at: http://127.0.0.1:8080/#/experiments/579090426674828470/runs/4c1e57a6313346268402c62241c0e015
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/579090426674828470
🏃 View run LogisticRegression_v2__application_train_managed_outliers_mising_cols__study_run at: http://127.0.0.1:8080/#/experiments/579090426674828470/runs/d363fffcb68d43c7b19d7b9cf68d078f
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/579090426674828470


In [23]:
X_train, X_test, y_train, y_test, preproc = prepare_data(
    data[
        ["EXT_SOURCE_1", "EXT_SOURCE_2", "EXT_SOURCE_3", "DAYS_BIRTH", "TARGET"]
    ],
    target_column="TARGET",
    categorical_strategy="most_frequent",
    outlier_strategy="replace_with_nan",
    outlier_threshold=0.05,
    test_size=0.2,
    random_state=42,
    encode_categoricals=True,
    drop_missing_columns=True,
    missing_col_threshold=0.5,
    polynomial_features=True,
)


run_experiment(
    model_name="LogisticRegression_v2",
    model_class=LogisticRegression,
    param_space_fn=lr_param_space,
    dataset_name="application_train_poly_managed_outliers_mising_cols",
    X_train=X_train,
    X_test=X_test,
    y_train=y_train,
    y_test=y_test,
    preprocessor=preproc,
    cv=5,
    n_trials=30,
)


🧹 Dropping 1 column(s) with missing ratio ≥ 0.5:
 - EXT_SOURCE_1
Found 3 numerical features
Found 0 binary features
Found 0 categorical features


[I 2025-05-08 18:04:04,580] A new study created in memory with name: no-name-a9501c76-c901-4326-9705-4478a2b2c6aa
/Users/jeremy/code/ocr/p7_pret_a_depenser/venv/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/jeremy/code/ocr/p7_pret_a_depenser/venv/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/jeremy/code/ocr/p7_pret_a_depenser/venv/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/jeremy/code/ocr/p7_pret_a_depenser/venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to cont

🏃 View run classy-goose-795 at: http://127.0.0.1:8080/#/experiments/579090426674828470/runs/72e751d499ae4d9d894d8e4ebecbf207
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/579090426674828470


[I 2025-05-08 18:05:03,508] Trial 1 finished with value: 0.8070177191074539 and parameters: {'solver_penalty': 'lbfgs__l2', 'positive_class_weight': 38.41049077824202, 'classifier__max_iter': 280, 'classifier__tol': 0.00011962411244648284, 'classifier__C': 0.10706978679235937, 'threshold': 0.49}. Best is trial 1 with value: 0.8070177191074539.


🏃 View run melodic-rat-747 at: http://127.0.0.1:8080/#/experiments/579090426674828470/runs/6c894e39ba414429b29b4f130f029326
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/579090426674828470


...

🏃 View run thundering-bear-830 at: http://127.0.0.1:8080/#/experiments/579090426674828470/runs/e6e15d0732984e94b0c1e09cbbdd3f78
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/579090426674828470


/Users/jeremy/code/ocr/p7_pret_a_depenser/venv/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/jeremy/code/ocr/p7_pret_a_depenser/venv/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/jeremy/code/ocr/p7_pret_a_depenser/venv/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/jeremy/code/ocr/p7_pret_a_depenser/venv/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/jeremy/code/ocr/p7_pret_a_depenser/venv/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached w

🏃 View run learned-fowl-338 at: http://127.0.0.1:8080/#/experiments/579090426674828470/runs/76e319d8022a493ba1f4d0811a4abd77
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/579090426674828470


/Users/jeremy/code/ocr/p7_pret_a_depenser/venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/jeremy/code/ocr/p7_pret_a_depenser/venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/jeremy/code/ocr/p7_pret_a_depenser/venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier,

🏃 View run LogisticRegression_v2__application_train_poly_managed_outliers_mising_cols__study_run at: http://127.0.0.1:8080/#/experiments/579090426674828470/runs/bef7f1abb6fd43c0bed2be7aca0ebe2e
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/579090426674828470


In [24]:
data_feat_engineer = data.copy()

data_feat_engineer["CREDIT_INCOME_PERCENT"] = (
    data_feat_engineer["AMT_CREDIT"] / data_feat_engineer["AMT_INCOME_TOTAL"]
)
data_feat_engineer["ANNUITY_INCOME_PERCENT"] = (
    data_feat_engineer["AMT_ANNUITY"] / data_feat_engineer["AMT_INCOME_TOTAL"]
)
data_feat_engineer["CREDIT_TERM"] = (
    data_feat_engineer["AMT_ANNUITY"] / data_feat_engineer["AMT_CREDIT"]
)
data_feat_engineer["DAYS_EMPLOYED_PERCENT"] = (
    data_feat_engineer["DAYS_EMPLOYED"] / data_feat_engineer["DAYS_BIRTH"]
)

X_train, X_test, y_train, y_test, preproc = prepare_data(
    data_feat_engineer,
    target_column="TARGET",
    categorical_strategy="most_frequent",
    outlier_strategy="replace_with_nan",
    outlier_threshold=0.05,
    test_size=0.2,
    random_state=42,
    encode_categoricals=True,
    drop_missing_columns=True,
    missing_col_threshold=0.5,
    polynomial_features=False,
)


run_experiment(
    model_name="LogisticRegression_v2",
    model_class=LogisticRegression,
    param_space_fn=lr_param_space,
    dataset_name="application_train_feat_engineer_managed_outliers_mising_cols",
    X_train=X_train,
    X_test=X_test,
    y_train=y_train,
    y_test=y_test,
    preprocessor=preproc,
    cv=5,
    n_trials=30,
)


🧹 Dropping 41 column(s) with missing ratio ≥ 0.5:
 - OWN_CAR_AGE
 - EXT_SOURCE_1
 - APARTMENTS_AVG
 - BASEMENTAREA_AVG
 - YEARS_BUILD_AVG
 - COMMONAREA_AVG
 - ELEVATORS_AVG
 - ENTRANCES_AVG
 - FLOORSMIN_AVG
 - LANDAREA_AVG
 - LIVINGAPARTMENTS_AVG
 - LIVINGAREA_AVG
 - NONLIVINGAPARTMENTS_AVG
 - NONLIVINGAREA_AVG
 - APARTMENTS_MODE
 - BASEMENTAREA_MODE
 - YEARS_BUILD_MODE
 - COMMONAREA_MODE
 - ELEVATORS_MODE
 - ENTRANCES_MODE
 - FLOORSMIN_MODE
 - LANDAREA_MODE
 - LIVINGAPARTMENTS_MODE
 - LIVINGAREA_MODE
 - NONLIVINGAPARTMENTS_MODE
 - NONLIVINGAREA_MODE
 - APARTMENTS_MEDI
 - BASEMENTAREA_MEDI
 - YEARS_BUILD_MEDI
 - COMMONAREA_MEDI
 - ELEVATORS_MEDI
 - ENTRANCES_MEDI
 - FLOORSMIN_MEDI
 - LANDAREA_MEDI
 - LIVINGAPARTMENTS_MEDI
 - LIVINGAREA_MEDI
 - NONLIVINGAPARTMENTS_MEDI
 - NONLIVINGAREA_MEDI
 - FONDKAPREMONT_MODE
 - HOUSETYPE_MODE
 - WALLSMATERIAL_MODE
Found 71 numerical features
Found 4 binary features
Found 9 categorical features


[I 2025-05-08 18:14:23,779] A new study created in memory with name: no-name-9dad4dcb-d9fc-4326-8808-26b26bcdb34f
/Users/jeremy/code/ocr/p7_pret_a_depenser/venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 2.
  warnings.warn(
/Users/jeremy/code/ocr/p7_pret_a_depenser/venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 2.
  warnings.warn(
/Users/jeremy/code/ocr/p7_pret_a_depenser/venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 2.
  warnings.warn(
/Users/jeremy/code/ocr/p7_pret_a_depenser/venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning: 'n_jobs' > 1 does not have any effect wh

🏃 View run colorful-dolphin-874 at: http://127.0.0.1:8080/#/experiments/579090426674828470/runs/2b768a26e624483c8dca1566fd2db710
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/579090426674828470


/Users/jeremy/code/ocr/p7_pret_a_depenser/venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 2.
  warnings.warn(
/Users/jeremy/code/ocr/p7_pret_a_depenser/venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 2.
  warnings.warn(
/Users/jeremy/code/ocr/p7_pret_a_depenser/venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 2.
  warnings.warn(
/Users/jeremy/code/ocr/p7_pret_a_depenser/venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 2.
  warnings.warn(
/Users/jeremy/code/ocr/p7_pret_a_depenser/ve

🏃 View run incongruous-cat-483 at: http://127.0.0.1:8080/#/experiments/579090426674828470/runs/5a6ee9f7c30a4262bf81a581b06c9560
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/579090426674828470


...

🏃 View run unruly-conch-299 at: http://127.0.0.1:8080/#/experiments/579090426674828470/runs/1d7324672d254a479db75e30d541aee7
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/579090426674828470


/Users/jeremy/code/ocr/p7_pret_a_depenser/venv/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/jeremy/code/ocr/p7_pret_a_depenser/venv/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/jeremy/code/ocr/p7_pret_a_depenser/venv/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/jeremy/code/ocr/p7_pret_a_depenser/venv/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/jeremy/code/ocr/p7_pret_a_depenser/venv/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached w

🏃 View run capricious-worm-399 at: http://127.0.0.1:8080/#/experiments/579090426674828470/runs/6603d757d10748948f76e5eba883f81a
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/579090426674828470


/Users/jeremy/code/ocr/p7_pret_a_depenser/venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 2.
  warnings.warn(
/Users/jeremy/code/ocr/p7_pret_a_depenser/venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 2.
  warnings.warn(
/Users/jeremy/code/ocr/p7_pret_a_depenser/venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 2.
  warnings.warn(
/Users/jeremy/code/ocr/p7_pret_a_depenser/venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 2.
  warnings.warn(
/Users/jeremy/code/ocr/p7_pret_a_depenser/ve

🏃 View run dazzling-goat-372 at: http://127.0.0.1:8080/#/experiments/579090426674828470/runs/4ea5f0f8dec44c82911d2d365ad69288
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/579090426674828470
🏃 View run LogisticRegression_v2__application_train_feat_engineer_managed_outliers_mising_cols__study_run at: http://127.0.0.1:8080/#/experiments/579090426674828470/runs/c30992d21a664cfdbb235289e4df351b
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/579090426674828470


## Random Forest

In [25]:
def random_forest_param_space(
    trial: optuna.trial.Trial,
    class_weight_range: Tuple[float, float],
    n_estimators_range: Tuple[int, int],
    max_depth_range: Tuple[int, int],
    min_samples_split_range: Tuple[int, int],
    min_samples_leaf_range: Tuple[int, int],
    max_features_range: Tuple,
    bootstrap_range: Tuple,
) -> Dict[str, Any]:
    """
    Suggests a hyperparameter configuration for a RandomForestClassifier using Optuna.

    Args:
        trial: Optuna trial object used to sample hyperparameters.
        class_weight_range: Range for the positive class weight (e.g., (1, 100)).
        n_estimators_range: Range for the number of trees.
        max_depth_range: Range for tree depth.
        min_samples_split_range: Range for the minimum number of samples to split an internal node.
        min_samples_leaf_range: Range for the minimum number of samples at a leaf node.

    Returns:
        A dictionary of hyperparameters for a scikit-learn Pipeline.
    """
    pos_weight = trial.suggest_float(
        "positive_class_weight", *class_weight_range
    )
    class_weight = {0: 1, 1: pos_weight}

    params = {
        "classifier__n_estimators": trial.suggest_int(
            "classifier__n_estimators", *n_estimators_range
        ),
        "classifier__max_depth": trial.suggest_int(
            "classifier__max_depth", *max_depth_range
        ),
        "classifier__min_samples_split": trial.suggest_int(
            "classifier__min_samples_split", *min_samples_split_range
        ),
        "classifier__min_samples_leaf": trial.suggest_int(
            "classifier__min_samples_leaf", *min_samples_leaf_range
        ),
        "classifier__max_features": trial.suggest_categorical(
            "classifier__max_features", max_features_range
        ),
        "classifier__bootstrap": trial.suggest_categorical(
            "classifier__bootstrap", bootstrap_range
        ),
        "classifier__class_weight": class_weight,
    }
    return params


rf_param_space = partial(
    random_forest_param_space,
    class_weight_range=(1, 100),
    n_estimators_range=(20, 100),
    max_depth_range=(3, 8),
    min_samples_split_range=(2, 10),
    min_samples_leaf_range=(1, 10),
    max_features_range=["sqrt"],
    bootstrap_range=[True],
)


In [26]:
X_train, X_test, y_train, y_test, preproc = prepare_data(
    data,
    target_column="TARGET",
    categorical_strategy="most_frequent",
    outlier_strategy="replace_with_nan",
    outlier_threshold=0.05,
    test_size=0.2,
    random_state=42,
    encode_categoricals=True,
    drop_missing_columns=True,
    missing_col_threshold=0.5,
    polynomial_features=False,
)

run_experiment(
    model_name="RandomForestClassifier_v2",
    model_class=RandomForestClassifier,
    param_space_fn=rf_param_space,
    dataset_name="application_train_managed_outliers_mising_cols",
    X_train=X_train,
    X_test=X_test,
    y_train=y_train,
    y_test=y_test,
    preprocessor=preproc,
    cv=5,
    n_trials=30,
)


🧹 Dropping 41 column(s) with missing ratio ≥ 0.5:
 - OWN_CAR_AGE
 - EXT_SOURCE_1
 - APARTMENTS_AVG
 - BASEMENTAREA_AVG
 - YEARS_BUILD_AVG
 - COMMONAREA_AVG
 - ELEVATORS_AVG
 - ENTRANCES_AVG
 - FLOORSMIN_AVG
 - LANDAREA_AVG
 - LIVINGAPARTMENTS_AVG
 - LIVINGAREA_AVG
 - NONLIVINGAPARTMENTS_AVG
 - NONLIVINGAREA_AVG
 - APARTMENTS_MODE
 - BASEMENTAREA_MODE
 - YEARS_BUILD_MODE
 - COMMONAREA_MODE
 - ELEVATORS_MODE
 - ENTRANCES_MODE
 - FLOORSMIN_MODE
 - LANDAREA_MODE
 - LIVINGAPARTMENTS_MODE
 - LIVINGAREA_MODE
 - NONLIVINGAPARTMENTS_MODE
 - NONLIVINGAREA_MODE
 - APARTMENTS_MEDI
 - BASEMENTAREA_MEDI
 - YEARS_BUILD_MEDI
 - COMMONAREA_MEDI
 - ELEVATORS_MEDI
 - ENTRANCES_MEDI
 - FLOORSMIN_MEDI
 - LANDAREA_MEDI
 - LIVINGAPARTMENTS_MEDI
 - LIVINGAREA_MEDI
 - NONLIVINGAPARTMENTS_MEDI
 - NONLIVINGAREA_MEDI
 - FONDKAPREMONT_MODE
 - HOUSETYPE_MODE
 - WALLSMATERIAL_MODE
Found 67 numerical features
Found 4 binary features
Found 9 categorical features


2025/05/08 19:17:58 INFO mlflow.tracking.fluent: Experiment with name 'RandomForestClassifier_v2' does not exist. Creating a new experiment.
[I 2025-05-08 19:17:58,544] A new study created in memory with name: no-name-9df3b84a-c81b-4e16-b6f0-5258c9aef4e8
[I 2025-05-08 19:18:32,891] Trial 0 finished with value: 0.8161458163978155 and parameters: {'positive_class_weight': 73.158598707212, 'classifier__n_estimators': 92, 'classifier__max_depth': 4, 'classifier__min_samples_split': 5, 'classifier__min_samples_leaf': 1, 'classifier__max_features': 'sqrt', 'classifier__bootstrap': True, 'threshold': 0.12000000000000001}. Best is trial 0 with value: 0.8161458163978155.


🏃 View run enchanting-finch-839 at: http://127.0.0.1:8080/#/experiments/928276634314426691/runs/298076c30fef4ac697ccc9006c1d33de
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/928276634314426691


[I 2025-05-08 19:18:54,988] Trial 1 finished with value: 0.8161458163978155 and parameters: {'positive_class_weight': 17.021426650421997, 'classifier__n_estimators': 46, 'classifier__max_depth': 3, 'classifier__min_samples_split': 7, 'classifier__min_samples_leaf': 8, 'classifier__max_features': 'sqrt', 'classifier__bootstrap': True, 'threshold': 0.33}. Best is trial 0 with value: 0.8161458163978155.


🏃 View run agreeable-elk-867 at: http://127.0.0.1:8080/#/experiments/928276634314426691/runs/52ff586c8ca64435a7f8f722f599f8dd
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/928276634314426691


...

🏃 View run dashing-roo-562 at: http://127.0.0.1:8080/#/experiments/928276634314426691/runs/14b36aa18286416c952f977b4f984dbf
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/928276634314426691


[I 2025-05-08 19:33:22,935] Trial 28 finished with value: 0.8161458163978155 and parameters: {'positive_class_weight': 45.00637099105667, 'classifier__n_estimators': 63, 'classifier__max_depth': 8, 'classifier__min_samples_split': 2, 'classifier__min_samples_leaf': 3, 'classifier__max_features': 'sqrt', 'classifier__bootstrap': True, 'threshold': 0.19}. Best is trial 0 with value: 0.8161458163978155.


🏃 View run capricious-squid-244 at: http://127.0.0.1:8080/#/experiments/928276634314426691/runs/bec306a22fe9442b809b30d81b05ff1d
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/928276634314426691


/Users/jeremy/code/ocr/p7_pret_a_depenser/venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/jeremy/code/ocr/p7_pret_a_depenser/venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/jeremy/code/ocr/p7_pret_a_depenser/venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metr

🏃 View run fun-gnu-355 at: http://127.0.0.1:8080/#/experiments/928276634314426691/runs/da9046fb24f349c2af1dde73d6bcf251
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/928276634314426691
🏃 View run RandomForestClassifier_v2__application_train_managed_outliers_mising_cols__study_run at: http://127.0.0.1:8080/#/experiments/928276634314426691/runs/85b718324b97472788ca2033bb6f2a51
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/928276634314426691


In [27]:
X_train, X_test, y_train, y_test, preproc = prepare_data(
    data[
        ["EXT_SOURCE_1", "EXT_SOURCE_2", "EXT_SOURCE_3", "DAYS_BIRTH", "TARGET"]
    ],
    target_column="TARGET",
    categorical_strategy="most_frequent",
    outlier_strategy="replace_with_nan",
    outlier_threshold=0.05,
    test_size=0.2,
    random_state=42,
    encode_categoricals=True,
    drop_missing_columns=True,
    missing_col_threshold=0.5,
    polynomial_features=True,
)


run_experiment(
    model_name="RandomForestClassifier_v2",
    model_class=RandomForestClassifier,
    param_space_fn=rf_param_space,
    dataset_name="application_train_poly_managed_outliers_mising_cols",
    X_train=X_train,
    X_test=X_test,
    y_train=y_train,
    y_test=y_test,
    preprocessor=preproc,
    cv=5,
    n_trials=30,
)


🧹 Dropping 1 column(s) with missing ratio ≥ 0.5:
 - EXT_SOURCE_1
Found 3 numerical features
Found 0 binary features
Found 0 categorical features


[I 2025-05-08 19:34:04,911] A new study created in memory with name: no-name-f474e3ce-847f-46d7-96c2-e2c61cb3fa7c
[I 2025-05-08 19:34:22,529] Trial 0 finished with value: 0.8161458163978155 and parameters: {'positive_class_weight': 28.563206398980054, 'classifier__n_estimators': 41, 'classifier__max_depth': 3, 'classifier__min_samples_split': 9, 'classifier__min_samples_leaf': 9, 'classifier__max_features': 'sqrt', 'classifier__bootstrap': True, 'threshold': 0.27}. Best is trial 0 with value: 0.8161458163978155.


🏃 View run useful-rat-747 at: http://127.0.0.1:8080/#/experiments/928276634314426691/runs/26aae9f4180149b884aca1dbc35f7056
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/928276634314426691


[I 2025-05-08 19:35:02,947] Trial 1 finished with value: 0.8161464727958555 and parameters: {'positive_class_weight': 55.45954596578046, 'classifier__n_estimators': 49, 'classifier__max_depth': 7, 'classifier__min_samples_split': 9, 'classifier__min_samples_leaf': 2, 'classifier__max_features': 'sqrt', 'classifier__bootstrap': True, 'threshold': 0.18}. Best is trial 1 with value: 0.8161464727958555.


🏃 View run judicious-mouse-315 at: http://127.0.0.1:8080/#/experiments/928276634314426691/runs/a9b1a8e006a14b5ab050fb979814c034
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/928276634314426691


...

🏃 View run smiling-hound-989 at: http://127.0.0.1:8080/#/experiments/928276634314426691/runs/a3623cf7b96e401aa9f1d9e84f4673c2
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/928276634314426691


[I 2025-05-08 19:52:42,849] Trial 28 finished with value: 0.8161458163978155 and parameters: {'positive_class_weight': 28.345148479257556, 'classifier__n_estimators': 41, 'classifier__max_depth': 7, 'classifier__min_samples_split': 3, 'classifier__min_samples_leaf': 7, 'classifier__max_features': 'sqrt', 'classifier__bootstrap': True, 'threshold': 0.1}. Best is trial 26 with value: 0.8161474575757581.


🏃 View run fun-sheep-821 at: http://127.0.0.1:8080/#/experiments/928276634314426691/runs/3ca0156f7bb842f8a552cdee9c782746
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/928276634314426691


[I 2025-05-08 19:53:23,269] Trial 29 finished with value: 0.813271398072315 and parameters: {'positive_class_weight': 26.413794407089352, 'classifier__n_estimators': 44, 'classifier__max_depth': 8, 'classifier__min_samples_split': 4, 'classifier__min_samples_leaf': 9, 'classifier__max_features': 'sqrt', 'classifier__bootstrap': True, 'threshold': 0.27}. Best is trial 26 with value: 0.8161474575757581.


🏃 View run dapper-mare-39 at: http://127.0.0.1:8080/#/experiments/928276634314426691/runs/89132e1af0ed4a8883e293f76609f012
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/928276634314426691
🏃 View run RandomForestClassifier_v2__application_train_poly_managed_outliers_mising_cols__study_run at: http://127.0.0.1:8080/#/experiments/928276634314426691/runs/b80e4210d0424a91834e4d3396a55b60
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/928276634314426691


In [28]:
data_feat_engineer = data.copy()

data_feat_engineer["CREDIT_INCOME_PERCENT"] = (
    data_feat_engineer["AMT_CREDIT"] / data_feat_engineer["AMT_INCOME_TOTAL"]
)
data_feat_engineer["ANNUITY_INCOME_PERCENT"] = (
    data_feat_engineer["AMT_ANNUITY"] / data_feat_engineer["AMT_INCOME_TOTAL"]
)
data_feat_engineer["CREDIT_TERM"] = (
    data_feat_engineer["AMT_ANNUITY"] / data_feat_engineer["AMT_CREDIT"]
)
data_feat_engineer["DAYS_EMPLOYED_PERCENT"] = (
    data_feat_engineer["DAYS_EMPLOYED"] / data_feat_engineer["DAYS_BIRTH"]
)

X_train, X_test, y_train, y_test, preproc = prepare_data(
    data_feat_engineer,
    target_column="TARGET",
    categorical_strategy="most_frequent",
    outlier_strategy="replace_with_nan",
    outlier_threshold=0.05,
    test_size=0.2,
    random_state=42,
    encode_categoricals=True,
    drop_missing_columns=True,
    missing_col_threshold=0.5,
    polynomial_features=False,
)


run_experiment(
    model_name="RandomForestClassifier_v2",
    model_class=RandomForestClassifier,
    param_space_fn=rf_param_space,
    dataset_name="application_train_feat_engineer_managed_outliers_mising_cols",
    X_train=X_train,
    X_test=X_test,
    y_train=y_train,
    y_test=y_test,
    preprocessor=preproc,
    cv=5,
    n_trials=30,
)


🧹 Dropping 41 column(s) with missing ratio ≥ 0.5:
 - OWN_CAR_AGE
 - EXT_SOURCE_1
 - APARTMENTS_AVG
 - BASEMENTAREA_AVG
 - YEARS_BUILD_AVG
 - COMMONAREA_AVG
 - ELEVATORS_AVG
 - ENTRANCES_AVG
 - FLOORSMIN_AVG
 - LANDAREA_AVG
 - LIVINGAPARTMENTS_AVG
 - LIVINGAREA_AVG
 - NONLIVINGAPARTMENTS_AVG
 - NONLIVINGAREA_AVG
 - APARTMENTS_MODE
 - BASEMENTAREA_MODE
 - YEARS_BUILD_MODE
 - COMMONAREA_MODE
 - ELEVATORS_MODE
 - ENTRANCES_MODE
 - FLOORSMIN_MODE
 - LANDAREA_MODE
 - LIVINGAPARTMENTS_MODE
 - LIVINGAREA_MODE
 - NONLIVINGAPARTMENTS_MODE
 - NONLIVINGAREA_MODE
 - APARTMENTS_MEDI
 - BASEMENTAREA_MEDI
 - YEARS_BUILD_MEDI
 - COMMONAREA_MEDI
 - ELEVATORS_MEDI
 - ENTRANCES_MEDI
 - FLOORSMIN_MEDI
 - LANDAREA_MEDI
 - LIVINGAPARTMENTS_MEDI
 - LIVINGAREA_MEDI
 - NONLIVINGAPARTMENTS_MEDI
 - NONLIVINGAREA_MEDI
 - FONDKAPREMONT_MODE
 - HOUSETYPE_MODE
 - WALLSMATERIAL_MODE
Found 71 numerical features
Found 4 binary features
Found 9 categorical features


[I 2025-05-08 19:53:41,739] A new study created in memory with name: no-name-11c043b4-b394-4339-a72e-89d84a69c44a
/Users/jeremy/code/ocr/p7_pret_a_depenser/venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/jeremy/code/ocr/p7_pret_a_depenser/venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/jeremy/code/ocr/p7_pret_a_depenser/venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no

🏃 View run auspicious-cub-656 at: http://127.0.0.1:8080/#/experiments/928276634314426691/runs/3188849f5ea94abbbdba4b801bcdcd6d
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/928276634314426691


[I 2025-05-08 19:54:42,539] Trial 1 finished with value: 0.8161458163978155 and parameters: {'positive_class_weight': 67.25979081658693, 'classifier__n_estimators': 64, 'classifier__max_depth': 6, 'classifier__min_samples_split': 2, 'classifier__min_samples_leaf': 7, 'classifier__max_features': 'sqrt', 'classifier__bootstrap': True, 'threshold': 0.45000000000000007}. Best is trial 1 with value: 0.8161458163978155.


🏃 View run monumental-gull-730 at: http://127.0.0.1:8080/#/experiments/928276634314426691/runs/6f4ce8c82bbe468e80f3b8949dcb6fcf
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/928276634314426691


...

🏃 View run upset-mare-466 at: http://127.0.0.1:8080/#/experiments/928276634314426691/runs/054438f47c494f40abfae14f7fad0a86
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/928276634314426691


[I 2025-05-08 20:12:40,033] Trial 29 finished with value: 0.8161458163978155 and parameters: {'positive_class_weight': 87.44184469478907, 'classifier__n_estimators': 94, 'classifier__max_depth': 7, 'classifier__min_samples_split': 5, 'classifier__min_samples_leaf': 10, 'classifier__max_features': 'sqrt', 'classifier__bootstrap': True, 'threshold': 0.4}. Best is trial 24 with value: 0.8161809437082757.


🏃 View run funny-ape-717 at: http://127.0.0.1:8080/#/experiments/928276634314426691/runs/8abcf86da94d4b0ea2b0ea96aba1792e
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/928276634314426691


/Users/jeremy/code/ocr/p7_pret_a_depenser/venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/jeremy/code/ocr/p7_pret_a_depenser/venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/jeremy/code/ocr/p7_pret_a_depenser/venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier,

🏃 View run RandomForestClassifier_v2__application_train_feat_engineer_managed_outliers_mising_cols__study_run at: http://127.0.0.1:8080/#/experiments/928276634314426691/runs/eb40959099a04b92a6e71690f586f362
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/928276634314426691


## LightGBM

In [32]:
def lightgbm_param_space(
    trial: optuna.trial.Trial,
    class_weight_range: Tuple[float, float],
    n_estimators_range: Tuple[int, int],
    learning_rate_range: Tuple[float, float],
    num_leaves_range: Tuple[int, int],
    max_depth_range: Tuple[int, int],
    min_child_samples_range: Tuple[int, int],
    subsample_range: Tuple[float, float],
    colsample_bytree_range: Tuple[float, float],
    reg_alpha_range: Tuple[float, float],
    reg_lambda_range: Tuple[float, float],
) -> Dict[str, Any]:
    """
    Suggests a hyperparameter configuration for an LGBMClassifier using Optuna.

    Args:
        trial: Optuna trial object used to sample hyperparameters.
        All other arguments define the range of hyperparameters.

    Returns:
        A dictionary of hyperparameters for a scikit-learn Pipeline.
    """
    pos_weight = trial.suggest_float(
        "positive_class_weight", *class_weight_range
    )
    class_weight = {0: 1, 1: pos_weight}

    params = {
        "classifier__n_estimators": trial.suggest_int(
            "classifier__n_estimators", *n_estimators_range
        ),
        "classifier__learning_rate": trial.suggest_float(
            "classifier__learning_rate", *learning_rate_range, log=True
        ),
        "classifier__num_leaves": trial.suggest_int(
            "classifier__num_leaves", *num_leaves_range
        ),
        "classifier__max_depth": trial.suggest_int(
            "classifier__max_depth", *max_depth_range
        ),
        "classifier__min_child_samples": trial.suggest_int(
            "classifier__min_child_samples", *min_child_samples_range
        ),
        "classifier__subsample": trial.suggest_float(
            "classifier__subsample", *subsample_range
        ),
        "classifier__colsample_bytree": trial.suggest_float(
            "classifier__colsample_bytree", *colsample_bytree_range
        ),
        "classifier__reg_alpha": trial.suggest_float(
            "classifier__reg_alpha", *reg_alpha_range
        ),
        "classifier__reg_lambda": trial.suggest_float(
            "classifier__reg_lambda", *reg_lambda_range
        ),
        "classifier__class_weight": class_weight,
    }
    return params


lgb_param_space = partial(
    lightgbm_param_space,
    class_weight_range=(1, 100),
    n_estimators_range=(50, 150),
    learning_rate_range=(0.01, 0.2),
    num_leaves_range=(20, 50),
    max_depth_range=(3, 6),
    min_child_samples_range=(10, 50),
    subsample_range=(0.6, 1.0),
    colsample_bytree_range=(0.6, 1.0),
    reg_alpha_range=(0.0, 1.0),
    reg_lambda_range=(0.0, 1.0),
)


In [36]:
X_train, X_test, y_train, y_test, preproc = prepare_data(
    data,
    target_column="TARGET",
    categorical_strategy="most_frequent",
    outlier_strategy="replace_with_nan",
    outlier_threshold=0.05,
    test_size=0.2,
    random_state=42,
    encode_categoricals=True,
    drop_missing_columns=True,
    missing_col_threshold=0.4,
    polynomial_features=True,
)


run_experiment(
    model_name="LGBMClassifier_v2",
    model_class=lgb.LGBMClassifier,
    param_space_fn=lgb_param_space,
    dataset_name="application_train_managed_outliers_mising_cols",
    X_train=X_train,
    X_test=X_test,
    y_train=y_train,
    y_test=y_test,
    preprocessor=preproc,
    cv=5,
    n_trials=10,
)


🧹 Dropping 49 column(s) with missing ratio ≥ 0.4:
 - OWN_CAR_AGE
 - EXT_SOURCE_1
 - APARTMENTS_AVG
 - BASEMENTAREA_AVG
 - YEARS_BEGINEXPLUATATION_AVG
 - YEARS_BUILD_AVG
 - COMMONAREA_AVG
 - ELEVATORS_AVG
 - ENTRANCES_AVG
 - FLOORSMAX_AVG
 - FLOORSMIN_AVG
 - LANDAREA_AVG
 - LIVINGAPARTMENTS_AVG
 - LIVINGAREA_AVG
 - NONLIVINGAPARTMENTS_AVG
 - NONLIVINGAREA_AVG
 - APARTMENTS_MODE
 - BASEMENTAREA_MODE
 - YEARS_BEGINEXPLUATATION_MODE
 - YEARS_BUILD_MODE
 - COMMONAREA_MODE
 - ELEVATORS_MODE
 - ENTRANCES_MODE
 - FLOORSMAX_MODE
 - FLOORSMIN_MODE
 - LANDAREA_MODE
 - LIVINGAPARTMENTS_MODE
 - LIVINGAREA_MODE
 - NONLIVINGAPARTMENTS_MODE
 - NONLIVINGAREA_MODE
 - APARTMENTS_MEDI
 - BASEMENTAREA_MEDI
 - YEARS_BEGINEXPLUATATION_MEDI
 - YEARS_BUILD_MEDI
 - COMMONAREA_MEDI
 - ELEVATORS_MEDI
 - ENTRANCES_MEDI
 - FLOORSMAX_MEDI
 - FLOORSMIN_MEDI
 - LANDAREA_MEDI
 - LIVINGAPARTMENTS_MEDI
 - LIVINGAREA_MEDI
 - NONLIVINGAPARTMENTS_MEDI
 - NONLIVINGAREA_MEDI
 - FONDKAPREMONT_MODE
 - HOUSETYPE_MODE
 - TOTALARE

2025/05/08 23:52:14 INFO mlflow.tracking.fluent: Experiment with name 'LGBMClassifier_v2' does not exist. Creating a new experiment.
[I 2025-05-08 23:52:14,208] A new study created in memory with name: no-name-0959e83b-ea37-4f12-9c5d-71b04e02a132


: 

In [31]:
# X_train, X_test, y_train, y_test, preproc = prepare_data(
#     data[
#         ["EXT_SOURCE_1", "EXT_SOURCE_2", "EXT_SOURCE_3", "DAYS_BIRTH", "TARGET"]
#     ],
#     target_column="TARGET",
#     categorical_strategy="most_frequent",
#     outlier_strategy="replace_with_nan",
#     outlier_threshold=0.05,
#     test_size=0.2,
#     random_state=42,
#     encode_categoricals=True,
#     drop_missing_columns=True,
#     missing_col_threshold=0.4,
#     polynomial_features=True,
# )


# run_experiment(
#     model_name="LGBMClassifier",
#     model_class=lgb.LGBMClassifier,
#     param_space_fn=lgb_param_space,
#     dataset_name="application_train_poly_managed_outliers_mising_cols",
#     X_train=X_train,
#     X_test=X_test,
#     y_train=y_train,
#     y_test=y_test,
#     preprocessor=preproc,
#     cv=5,
#     n_trials=30,
# )
